### Problem Statement

Use Case: Developing an LLM chatbot that would give product details

Requirements: 
*  Name of the product. 
*  Description of the product. 
*  Price of the product in USD

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import pprint

In [2]:
## Langchain and Langsmith tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_TRACING_V2"]="true"

## Getting Froq API key
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [3]:
llm = ChatGroq(model='gemma2-9b-it')
result = llm.invoke('Give me the details of the product iphone 13. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to 2 decimal place. Give the product details in the following format: Return a JSON object.')
print(result.content)

```json
{
  "name": "iPhone 13",
  "description": "The iPhone 13 features a Super Retina XDR display, A15 Bionic chip, and advanced camera systems. Its dual 12MP camera system captures stunning photos and videos with Cinematic mode. Enjoy all-day battery life and a sleek design with a durable ceramic shield front cover. Available in multiple colors and storage options.",
  "price": 699.00 
}
``` 


Please note that the price may vary depending on the retailer, storage capacity, and current promotions. 



In [12]:
llm_openai = ChatOpenAI(model = "o1-mini")
result = llm_openai.invoke('Give me the details of the product iphone 13. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to 2 decimal place. Give the product details in the following format: Return a JSON object.')
print(result.content)

```json
{
  "product_name": "iPhone 13",
  "description": "The iPhone 13 features a sleek design with a durable aluminum frame and Ceramic Shield front cover. It is powered by the A15 Bionic chip, delivering high performance and efficiency. The device boasts a dual-camera system with advanced computational photography, enabling stunning photos and videos. The Super Retina XDR display offers vibrant colors and brightness for an immersive viewing experience. With 5G connectivity, the iPhone 13 ensures faster downloads and streaming. It runs on iOS, providing seamless integration with Apple's ecosystem and access to a vast array of apps. The battery life is improved, offering all-day usage. Available in multiple colors, the iPhone 13 combines style, functionality, and cutting-edge technology.",
  "price_usd": 799.00
}
```


In [4]:
## Creating Product Pydantic Class
class Product_Details(BaseModel):
    name : str
    description : str
    price : float=Field(description="The price is in USD")

In [5]:
## Create the output parser
parser = JsonOutputParser(pydantic_object=Product_Details)


In [6]:
## Creating the prompts
prompt = PromptTemplate(
    template="Give me the details of the product {product_name}. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to 2 decimal place. Give the product details in {format_instruction}",
    input_variables=["product_name"],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)

input_variables=['product_name'] input_types={} partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}, "price": {"description": "The price is in USD", "title": "Price", "type": "number"}}, "required": ["name", "description", "price"]}\n```'} template='Give me the details of the product {product_name}. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to

In [7]:
chain = prompt | llm | parser

In [16]:
response = chain.invoke({"product_name":"Motorola Moto g64 256 GB"})
pprint.pprint(response)

{'description': 'The Motorola Moto g64 256 GB is a smartphone featuring a '
                'powerful processor, ample storage, and a vibrant display. Its '
                'long-lasting battery and advanced camera system make it a '
                'great choice for everyday use.',
 'name': 'Motorola Moto g64 256 GB',
 'price': 299.99}


In [14]:
chain2 = prompt | llm_openai | parser

In [17]:
response = chain2.invoke({"product_name":"Motorola Moto g64 256 GB"})
pprint.pprint(response)

{'description': 'The Motorola Moto g64 256 GB combines sleek design with '
                'impressive performance. It features a 6.5-inch Full HD+ '
                'display for vibrant visuals and a smooth viewing experience. '
                'Powered by a powerful octa-core processor and 6GB of RAM, the '
                'phone handles multitasking and gaming with ease. With 256 GB '
                'of internal storage, you have ample space for apps, photos, '
                'and videos. The device is equipped with a versatile '
                'triple-camera system, including a 64MP main sensor, enabling '
                'high-quality photography and video recording. A robust '
                '5000mAh battery ensures long-lasting usage, while fast '
                'charging support keeps you powered up quickly. Running on the '
                'latest Android version, the Moto g64 offers a user-friendly '
                'interface and enhanced security features, making it a 